<a href="https://colab.research.google.com/github/victorgrrtj/trabalho_final_puc/blob/main/trabalho_final_treino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Trabalho Final - Treinamento do Modelo Preditivo

## Importação de Bibliotecas

In [1]:
# Importação das bibliotecas
import requests
import pandas as pd
import re
from IPython.display import HTML
import math
import time
import random
import datetime
import joblib
headers = {
    'Authorization': 'Bearer $ACCESS_TOKEN',
}

## Sumarização de todas as extrações.

In [2]:
""" Contém looping para todas as páginas.
Retorna um dicionário com as informações de cada imóvel.
Devido à quantidade de páginas, este processo pode demorar mais de 30 minutos."""

headers = {
    'Authorization': 'Bearer $ACCESS_TOKEN',
}

offset = 0
pag_inicial = 'https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset='+str(offset)

response = requests.get(
  pag_inicial,
  headers=headers)
json = response.json()
qtd_pag = round(json['paging']['total']/50)
qtd_itens = json['paging']['total']

print("Foram encontradas {} páginas e {} produtos.".format(qtd_pag, qtd_itens))

offset = 0
url_list = [pag_inicial]
for page in range(qtd_pag):
  offset += 50
  prox_pagina = 'https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset='+str(offset)
  url_list.append(prox_pagina)

extracted_data = []

for index, url in enumerate(url_list):
  response = requests.get(
      url,
      headers=headers)
  print("Codigo de resposta da pagina {}: {}".format(url, response.status_code))

  json = response.json()
  results = json['results']
  time.sleep(random.randint(2,6))

  for item in results:
    product_dict = {}
    product_dict['id'] = item.get('id')
    product_dict['titulo'] = item.get('title')
    product_dict['condicao'] = item.get('condition')
    product_dict['link'] = item.get('permalink')
    product_dict['preco'] = item.get('price')
    for attribute in item.get('attributes'):
      if attribute.get('name') == 'Marca':
        product_dict['marca'] = attribute.get('value_name')
      elif attribute.get('name') == 'Linha':
        product_dict['linha'] = attribute.get('value_name')
      elif attribute.get('name') == 'Modelo':
        product_dict['modelo'] = attribute.get('value_name')
    product_dict['nome_vendedor'] = item.get('seller').get('nickname')
    product_dict['nvl_vendedor'] = item.get('seller').get('seller_reputation').get('level_id')
    product_dict['vendas_concluidas'] = item.get('seller').get('seller_reputation').get('transactions').get('completed')
    product_dict['qtd_avaliacoes'] = item.get('seller').get('seller_reputation').get('transactions').get('total')
    product_dict['avlc_pos'] = item.get('seller').get('seller_reputation').get('transactions').get('ratings').get('positive')
    product_dict['avlc_neut'] = item.get('seller').get('seller_reputation').get('transactions').get('ratings').get('neutral')
    product_dict['avlc_neg'] = item.get('seller').get('seller_reputation').get('transactions').get('ratings').get('negative')
    extracted_data.append(product_dict)

  print('Finished page ' + str(index + 1))
print('Finished all pages')

Foram encontradas 17 páginas e 838 produtos.
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=0: 200
Finished page 1
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=50: 200
Finished page 2
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=100: 200
Finished page 3
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=150: 200
Finished page 4
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=200: 200
Finished page 5
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=250: 200
Finished page 6
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=300: 200
Finished page 7
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=350: 200
Finish

In [21]:
df = pd.json_normalize(extracted_data)

In [22]:
df["modelo"].unique()

array(['iPhone 14', 'iPhone 14 Pro Max', 'iPhone 13', 'iPhone 14 Pro',
       '(n/d)', 'iPhone 13 Pro', 'iPhone 7 Plus', 'IPHONE 14', 'iPhone 7',
       'iPhone 14 BR', 'iPhone 14 Plus', 'iPhone 14 pro', '14 PRO MAX',
       'iPhone 11', '14', 'IPHONE 14 PRO',
       'iPhone 14 128Gb Novo Lacrado Garantia de 1 Ano',
       'Iphone 14 128gb', 'iPhone',
       'iPhone 14 128Gb - Novo Lacrado - Garantia de 1 Ano',
       'iPhone 14Pro Max 1TB', '14 Pro 256gb', '14 PRO', 'Apple',
       'Iphone 14 plus 128gb', 'iPhone 14 PRO MAX', 'iPhone 14 PRO',
       'Tela 6,7',
       'iPhone 14 Pro Apple iPhone 14pro iphone 13 iphone 13 pro iphone12pro celular Apple iPhone 14pro',
       'IPhone 14', 'iPhone 14 128gb', 'iphone 14 pro max128',
       'iPhone 14 pro 128 gb', '14 Pro', 'iPhone 14 Pro Max (128 GB)',
       '14 pro', 'iphone 14pro max 256gb', 'Iphone 14 Pro 256gb',
       'iPhone 14 Pro iphone13pro iPhone 13 iphone13pro iphone11 iPhone 11 pro Apple Celular Roxo',
       'Iphone 14 pro max

In [23]:
df['modelo'] = df['modelo'].str.replace('IPHONE', 'iPhone')
df['modelo'] = df['modelo'].str.replace('iphone', 'iPhone')
df['modelo'] = df['modelo'].str.replace('Iphone', 'iPhone')
df['modelo'] = df['modelo'].str.replace('IPhone', 'iPhone')

In [24]:
df.drop(df.index[df['modelo'] == 'iPhone'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 7'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 7 Plus'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 12 Pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhones 8 ao 14'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 11'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 11 Pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 11 Pro Max'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 13'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 13 Pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 13 Pro Max'], inplace=True)
df.drop(df.index[df['modelo'] == '(n/d)'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 14 Pro Apple iPhone 14pro iPhone 13 iPhone 13 pro iPhone12pro celular Apple iPhone 14pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 14 Pro iPhone13pro iPhone 13 iPhone13pro iPhone11 iPhone 11 pro Apple Celular Roxo'], inplace=True)
df.drop(df.index[df['modelo'] == 'Apple'], inplace=True)
df.drop(df.index[df['modelo'] == 'Tela 6,7'], inplace=True)
df.drop(df.index[df['titulo'].str.contains('iphone', case=False) == False], inplace=True)
df.reset_index(inplace=True, drop=True)

In [25]:
df.loc[df["modelo"] == "14 PRO MAX", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "14 PRO MAX 128GB", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "14 Pro 256gb", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14 PRO ROXO PROFUNDO 128GB", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "14 PRO", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14Pro Max 1TB", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "Sem Rádio", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "14 Pro Max", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14Pro Tela de 6,1 polegadas¹", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone Pro Max 256GB", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 Pro Max Original", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "A2894", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "MPX33BE/A", "modelo"] = 'iPhone 14'
df.loc[df["modelo"] == "iPhone 14 PRO", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14 PRO MAX", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 pro max", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 pro max128", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "14 Pro", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14pro max 256gb", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "IPhone 14 Pro Max", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 pro maxx", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "14 128 gb", "modelo"] = 'iPhone 14'
df.loc[df["modelo"] == "iPhone 11 Pro Max", "modelo"] = 'iPhone 11 Pro Max'
df.loc[df["modelo"] == "14 pro Max 128gb", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 por max", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 PRO MAX ROXO PROFUNDO 128GB", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "A2650", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14 128gb", "modelo"] = 'iPhone 14'
df.loc[df["modelo"] == "MQ503BE/A", "modelo"] = 'iPhone 14 Plus'
df.loc[df["modelo"] == "iPhone 14 BR", "modelo"] = 'iPhone 14'
df.loc[df["modelo"] == "14", "modelo"] = 'iPhone 14'
df.loc[df["modelo"] == "14 Pro Max 256gb", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "14 pro max 256gb", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "14 pro Max", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 Pro Max - NOVO sem a caixa", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 pro Max", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "14 Pro Max 256", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14s Pro Max", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "14 pro max", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 Pro MAX 128GB", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 Pro americano (esim)", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14 pro 128gb", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14 Pro 128", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14 Pro - Americano (esim)", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14 Pro Max 128gb (sem caixa)", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 Pro max 128 GB", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 pro", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "14 pro", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14 128Gb Novo Lacrado Garantia de 1 Ano", "modelo"] = 'iPhone 14'
df.loc[df["modelo"] == "iPhone 14 128Gb - Novo Lacrado - Garantia de 1 Ano", "modelo"] = 'iPhone 14'
df.loc[df["modelo"] == "iPhone 14 plus 128gb", "modelo"] = 'iPhone 14 Plus'
df.loc[df["modelo"] == "iPhone 14 pro 128 gb", "modelo"] = 'iPhone 14 Pro'
df.loc[df["modelo"] == "iPhone 14 Pro Max (128 GB)", "modelo"] = 'iPhone 14 Pro Max'
df.loc[df["modelo"] == "iPhone 14 Pro 256gb", "modelo"] = 'iPhone 14 Pro'

In [26]:
del_index = df[(df['modelo'] == 'iPhone 14 Pro Max') & (df['titulo'].str.contains('max', case=False) == False)].index
del_index = del_index.append(df[(df['modelo'] == 'iPhone 14 Pro Max') & (df['titulo'].str.contains('plus', case=False) == True)].index)
del_index = del_index.append(df[(df['modelo'] == 'iPhone 14 Pro Max') & (df['titulo'].str.contains('iphone 14', case=False) == False)].index)
del_index = del_index.append(df[(df['modelo'] == 'iPhone 14 Pro') & (df['titulo'].str.contains('pro', case=False) == False)].index)
del_index = del_index.append(df[(df['modelo'] == 'iPhone 14 Pro') & (df['titulo'].str.contains('max', case=False) == True)].index)
del_index = del_index.append(df[(df['modelo'] == 'iPhone 14 Pro') & (df['titulo'].str.contains('plus', case=False) == True)].index)
del_index = del_index.append(df[(df['modelo'] == 'iPhone 14 Pro') & (df['titulo'].str.contains('iphone 14', case=False) == False)].index)
del_index = del_index.append(df[(df['modelo'] == 'iPhone 14') & (df['titulo'].str.contains('pro', case=False) == True)].index)
del_index = del_index.append(df[(df['modelo'] == 'iPhone 14') & (df['titulo'].str.contains('max', case=False) == True)].index)
del_index = del_index.append(df[(df['modelo'] == 'iPhone 14') & (df['titulo'].str.contains('plus', case=False) == True)].index)
del_index = del_index.append(df[(df['modelo'] == 'iPhone 14') & (df['titulo'].str.contains('iphone 14', case=False) == False)].index)
del_index = del_index.append(df[df['titulo'].str.contains('iphone', case=False) == False].index)
del_index = del_index.append(df[df['titulo'].str.contains('14', case=False) == False].index)

In [27]:
df = df.drop(del_index).reset_index(drop=True)

In [28]:
df_base = df[['titulo', 'modelo']]

In [30]:
df_base.modelo.unique()

array(['iPhone 14', 'iPhone 14 Pro Max', 'iPhone 14 Pro',
       'iPhone 14 Plus'], dtype=object)

In [31]:
from sklearn.model_selection import train_test_split

# Separação do label e das features
X = df_base.drop('modelo', axis=1).values
y = df_base['modelo'].values

# Separação de dados de treino e teste
train_features, test_features, class_train, class_test = train_test_split(X, y, test_size=0.20, random_state=10, stratify=y)

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

# Criação de matriz com a contagem de cada token SEM stop words
cv = CountVectorizer(max_features=1000)
cv_train_features = cv.fit_transform(train_features.ravel())
cv_test_features = cv.transform(test_features.ravel())

print('Shape das features de treino:', cv_train_features.shape, ' Shape das features de teste:', cv_test_features.shape)

Shape das features de treino: (606, 200)  Shape das features de teste: (152, 200)


In [33]:
from sklearn import metrics

def train_predict_model(classifier, 
                        train_features, train_labels, 
                        test_features, test_labels):
    classifier = classifier.fit(train_features, train_labels)
    predictions = classifier.predict(test_features) 
    return predictions 

In [34]:
def display_confusion_matrix(true_labels, predicted_labels, classes=[1,0]):
    
    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm = metrics.confusion_matrix(y_true=true_labels, y_pred=predicted_labels, 
                                  labels=classes)

    cm_frame = pd.DataFrame(cm, index=classes, columns=classes,)
    cm_frame.index.name = 'Actual'
    cm_frame.columns.name = 'Predicted'

    print(cm_frame)

def display_classification_report(true_labels, predicted_labels, classes=[1,0]):

    report = metrics.classification_report(y_true=true_labels, 
                                           y_pred=predicted_labels, 
                                           labels=classes) 
    print(report)

def display_model_performance_metrics(true_labels, predicted_labels, classes=[1,0]):
    print('Model Performance metrics:')
    print('-'*30)
    get_metrics(true_labels=true_labels, predicted_labels=predicted_labels)
    print('\nModel Classification report:')
    print('-'*30)
    display_classification_report(true_labels=true_labels, predicted_labels=predicted_labels, 
                                  classes=classes)
    print('\nPrediction Confusion Matrix:')
    print('-'*30)
    display_confusion_matrix(true_labels=true_labels, predicted_labels=predicted_labels, 
                             classes=classes)
    
def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))


In [35]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

rfc = RandomForestClassifier(n_jobs=-1)

classes = list(df_base['modelo'].unique())

classifier = rfc.fit(cv_train_features, class_train)

rfc_tfidf_predictions = train_predict_model(classifier=rfc, 
                                                train_features=cv_train_features, train_labels=class_train,
                                                test_features=cv_test_features, test_labels=class_test)

display_model_performance_metrics(true_labels=class_test, predicted_labels=rfc_tfidf_predictions,classes=classes)

Model Performance metrics:
------------------------------
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Model Classification report:
------------------------------
                   precision    recall  f1-score   support

        iPhone 14       1.00      1.00      1.00        40
iPhone 14 Pro Max       1.00      1.00      1.00        70
    iPhone 14 Pro       1.00      1.00      1.00        33
   iPhone 14 Plus       1.00      1.00      1.00         9

         accuracy                           1.00       152
        macro avg       1.00      1.00      1.00       152
     weighted avg       1.00      1.00      1.00       152


Prediction Confusion Matrix:
------------------------------
Predicted          iPhone 14  iPhone 14 Pro Max  iPhone 14 Pro  iPhone 14 Plus
Actual                                                                        
iPhone 14                 40                  0              0               0
iPhone 14 Pro Max          0                 70      

In [36]:
joblib.dump(classifier, "model.joblib")
joblib.dump(cv, "cv.joblib")

['cv.joblib']